# Irbid weather data

This notebook will cleanse, visualize and explore weather data for Irbid city obtained from ArabiaWeather. For further details please check the technical documentation.

In [1]:
# To measure the loading time
import time

start_time = time.time()

# Reading the data

In [ ]:
# Install pyspark because weather data is big data and we need pyspark to deal with it
!pip install pyspark

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType,StructField, IntegerType, FloatType, StringType, DateType

# In this colab we didn't use cluster we used single machine local[*].
# However, if we want change it to cluster, we can change it to master url 7077
spark =  SparkSession.builder.appName("SparkApp").master("local[*]").getOrCreate()

In [ ]:
# Importing the irbid weather dataset
WEATHER_IRBID = "Irbid.csv"

# Using our own schema to define datatypes of the dataset features
Myschema = StructType([
StructField('Station', StringType(),True),
StructField('Date/Time', DateType(),True),
StructField('Air Dew Point', IntegerType(),True),
StructField('Air Temperature (OC)', IntegerType(),True),
StructField('Humidity %', IntegerType(),True),
StructField('Manual Present Weather', StringType(),True),
StructField('Cloud Type', StringType(),True),
StructField('Clouds Cover (Okta)', IntegerType(),True),
StructField('Cloud Cover %', IntegerType(),True),
StructField('Wind Direction (Degrees)', IntegerType(),True),
StructField('Wind Speed (MPS)', IntegerType(),True),
StructField('Wind Type', StringType(),True)])

# Reading the data in df
df = spark.read.csv(WEATHER_IRBID,header=True, schema=Myschema)

In [ ]:
# Show the df
df.show(10)

In [ ]:
# Show summary statistics
df.describe().show()

# Handling missing values

In [ ]:
# Showing the null values' count
from pyspark.sql.functions import *
col_null_cnt_df =  df.select([count(when(col(c).isNull(),c)).alias(c) for c in df.columns])
col_null_cnt_df.show()

In [ ]:
# Dropping features with high percentage of nulls and will not affect the agriculture highly
df = df.drop('Air Dew Point') # --> can be known from humidity
df = df.drop('Manual Present Weather') # --> the general weather, can be known from other features
df = df.drop('Cloud Type') # --> type doesn't affect production
df = df.drop('Clouds Cover (Okta)') # --> we have cloud cover in %

In [ ]:
# Drop all rows where any value at specific columns are NAs, because its not a big number of rows
df = df.na.drop(how='any', subset=['Air Temperature (OC)','Cloud Cover %', 'Wind Speed (MPS)', 'Wind Type']) # 'any' is the defult

In [ ]:
# Using the window technique to fill the null values in the wind direction column using the last non-null value within the window.

from pyspark.sql.functions import col, last
from pyspark.sql.window import Window

target_column = "Wind Direction (Degrees)"

# Sorting the DataFrame by the date column in ascending order
sorted_df = df.orderBy("Date/Time")

# Creating a window specification based on the date column
window_spec = Window.orderBy("Date/Time").rowsBetween(Window.unboundedPreceding, 0)

# Filling null values in the target column using the last non-null value
df = sorted_df.withColumn(target_column, last(col(target_column), ignorenulls=True).over(window_spec))

df.show()

In [ ]:
# Drop all rows where any value at specific columns are NAs, for any rows that didn't have a non vull value before them
df = df.na.drop(how='any', subset=['Wind Direction (Degrees)']) # 'any' is the defult

In [ ]:
# Show the nulls again
from pyspark.sql.functions import *
col_null_cnt_df =  df.select([count(when(col(c).isNull(),c)).alias(c) for c in df.columns])
col_null_cnt_df.show()

In [ ]:
# Using linear regression to create a formula that predicts the humidity from other features

from pyspark.sql.functions import col, isnan
from pyspark.ml.feature import StringIndexer, VectorAssembler
from pyspark.ml.regression import LinearRegression

# Filter out rows with null or NaN values in the Humidity column
data = df.filter(col("Humidity %").isNotNull() & ~isnan("Humidity %"))

# Create StringIndexer to convert the Station column into numerical labels
station_indexer = StringIndexer(inputCol="Station", outputCol="Station_Index")
data = station_indexer.fit(data).transform(data)

# Create StringIndexer to convert the Wind Type column into numerical labels
wind_type_indexer = StringIndexer(inputCol="Wind Type", outputCol="Wind_Type_Index")
data = wind_type_indexer.fit(data).transform(data)

# Include the Station_Index and Wind_Type_Index columns in the VectorAssembler
assembler = VectorAssembler(inputCols=["Air Temperature (OC)", "Cloud Cover %", "Wind Direction (Degrees)", "Wind Speed (MPS)"],
                            outputCol="features")
data = assembler.transform(data)

# Select the relevant columns for the linear regression model
selected_data = data.select("features", "Humidity %")

# Split the data into training and test sets
train_data, test_data = selected_data.randomSplit([0.8, 0.2], seed=42)

# Create and fit the linear regression model
lr = LinearRegression(featuresCol="features", labelCol="Humidity %")
model = lr.fit(train_data)


In [ ]:
from pyspark.ml.evaluation import RegressionEvaluator

predictions = model.transform(test_data)

# Evaluate the model using R-squared
evaluator = RegressionEvaluator(labelCol="Humidity %", predictionCol="prediction", metricName="r2")
r2 = evaluator.evaluate(predictions)

print("R-squared:", r2)

In [ ]:
# Getting the formula's intercept and coefficients
coefficients = model.coefficients
intercept = model.intercept

print(coefficients)
print(intercept)

In [ ]:
# Filling nulls in the humidity column with the predections from the formula

from pyspark.sql.functions import col, when

# Define the equation coefficients and intercept
coefficients = [-1.6125718631802315,0.14974536899814145,0.1012784149968023,-1.3526705169477602]
intercept = 55.77390283264818

# Fill null values in the "Humidity %" column using the equation
df = df.withColumn("Humidity %", when(col("Humidity %").isNull(),
                                      intercept + (coefficients[0] * col("Air Temperature (OC)")) +
                                      (coefficients[1] * col("Cloud Cover %")) +
                                      (coefficients[1] * col("Wind Direction (Degrees)")) +
                                      (coefficients[3] * col("Wind Speed (MPS)")))
                               .otherwise(col("Humidity %")))

# Show the updated DataFrame
df.show()

In [ ]:
# Checking the null count for the last time
from pyspark.sql.functions import *
col_null_cnt_df =  df.select([count(when(col(c).isNull(),c)).alias(c) for c in df.columns])
col_null_cnt_df.show()

In [ ]:
# Checking the data count
df.count()

# EDA and visualizations

In [ ]:
# Plotting the general weather among years

import plotly.graph_objects as go
import pandas as pd

features = ["Station", "Date/Time", "Air Temperature (OC)", "Humidity %", "Cloud Cover %", "Wind Direction (Degrees)", "Wind Speed (MPS)", "Wind Type"]

for feature in features:
    data = df.select('Date/Time', feature).toPandas()

    # Create a Plotly figure
    fig = go.Figure()

    # Add a scatter trace for the feature
    fig.add_trace(go.Scatter(x=data['Date/Time'], y=data[feature], mode='lines', name=feature))

    # Set the layout
    fig.update_layout(title=feature + ' Trends', xaxis_title='Date/Time', yaxis_title=feature)

    # Show the plot
    fig.show()


In [ ]:
# Dropping features that will not assess the prediction of agriculture production according to the subject matter expert
df = df.drop('Wind Type')
df = df.drop('Cloud Cover %')
df = df.drop('Wind Direction (Degrees)')

In [ ]:
# Shwoing the df
df.show()

In [ ]:
# Plotting the temperature for months amongst years

import plotly.express as px
import pandas as pd

# Convert the DataFrame to Pandas for easier manipulation
data_pd = df.toPandas()

# Extract the year and month from the Date/Time column
data_pd['Year'] = pd.to_datetime(data_pd['Date/Time']).dt.year
data_pd['Month'] = pd.to_datetime(data_pd['Date/Time']).dt.month

# Group the data by Year and Month and calculate the average temperature
average_temp = data_pd.groupby(['Year', 'Month'])['Air Temperature (OC)'].mean().reset_index()

# Create separate plots for each year
for year in average_temp['Year'].unique():
    year_data = average_temp[average_temp['Year'] == year]

    # Create the plot using Plotly Express
    fig = px.line(year_data, x='Month', y='Air Temperature (OC)')

    # Set the layout
    fig.update_layout(title=f'Average Air Temperature by Month - {year}',
                      xaxis_title='Month',
                      yaxis_title='Air Temperature (OC)')

    # Show the plot
    fig.show()


In [ ]:
# Plotting the humidity for months amongst years

import plotly.express as px
import pandas as pd

# Convert the DataFrame to Pandas for easier manipulation
data_pd = df.toPandas()

# Extract the year and month from the Date/Time column
data_pd['Year'] = pd.to_datetime(data_pd['Date/Time']).dt.year
data_pd['Month'] = pd.to_datetime(data_pd['Date/Time']).dt.month

# Group the data by Year and Month and calculate the average humidity
average_humidity = data_pd.groupby(['Year', 'Month'])['Humidity %'].mean().reset_index()

# Iterate over each unique year
for year in average_humidity['Year'].unique():
    year_data = average_humidity[average_humidity['Year'] == year]

    # Create the plot using Plotly Express
    fig = px.line(year_data, x='Month', y='Humidity %')

    # Set the layout
    fig.update_layout(title=f'Average Humidity by Month - {year}',
                      xaxis_title='Month',
                      yaxis_title='Humidity %')

    # Show the plot
    fig.show()


In [ ]:
# Plotting the wind speed for months amongst years

import plotly.express as px
import pandas as pd

# Convert the DataFrame to Pandas for easier manipulation
data_pd = df.toPandas()

# Extract the year and month from the Date/Time column
data_pd['Year'] = pd.to_datetime(data_pd['Date/Time']).dt.year
data_pd['Month'] = pd.to_datetime(data_pd['Date/Time']).dt.month

# Group the data by Year and Month and calculate the average wind speed
average_wind_speed = data_pd.groupby(['Year', 'Month'])['Wind Speed (MPS)'].mean().reset_index()

# Iterate over each unique year
for year in average_wind_speed['Year'].unique():
    year_data = average_wind_speed[average_wind_speed['Year'] == year]

    # Create the plot using Plotly Express
    fig = px.line(year_data, x='Month', y='Wind Speed (MPS)')

    # Set the layout
    fig.update_layout(title=f'Average Wind Speed by Month - {year}',
                      xaxis_title='Month',
                      yaxis_title='Wind Speed (MPS)')

    # Show the plot
    fig.show()


In [ ]:
# Plotting the temperature and humidity over time

import pandas as pd
import plotly.graph_objects as go

# Convert the Spark DataFrame to Pandas DataFrame
df_pandas = df.toPandas()

# Line Plot: Temperature and Humidity Over Time
fig = go.Figure()
fig.add_trace(go.Scatter(x=df_pandas['Date/Time'], y=df_pandas['Air Temperature (OC)'], name='Temperature'))
fig.add_trace(go.Scatter(x=df_pandas['Date/Time'], y=df_pandas['Humidity %'], name='Humidity'))
fig.update_layout(title='Temperature and Humidity Over Time', xaxis_title='Date/Time', yaxis_title='Value')
fig.show()

In [ ]:
# Plotting the distribution of temperature by month
df_pandas['Month'] = pd.to_datetime(df_pandas['Date/Time']).dt.month
fig = px.box(df_pandas, x='Month', y='Air Temperature (OC)', title='Distribution of Temperature by Month')
fig.update_layout(xaxis_title='Month', yaxis_title='Temperature (OC)')
fig.show()

# Data transformation

In [ ]:
# Aggregate hours of each day to days based on the mean value (to include the effect of night times)

from pyspark.sql.functions import mean
from pyspark.sql.functions import to_date

df = df.withColumn("Date", to_date(df["Date/Time"]))

df = df.groupBy("Date").agg(
    mean("Air Temperature (OC)").alias("Mean Air Temperature"),
    mean("Humidity %").alias("Mean Humidity"),
    mean("Wind Speed (MPS)").alias("Mean Wind Speed")
)

df.show()

In [ ]:
# Check the count after aggregation
df.count()

In [ ]:
# General weather among days again
import plotly.express as px

# Convert the PySpark DataFrame to a Pandas DataFrame
pandas_df = df.select("*").toPandas()

# Plotting the data using Plotly
fig = px.line(pandas_df, x="Date", y=["Mean Air Temperature", "Mean Humidity", "Mean Wind Speed"],
              labels={"value": "Value"}, title="Mean Measures")
fig.update_layout(xaxis_tickangle=-45)
fig.show()


In [ ]:
# Mean wind speed will also be dropped as its nearly not changing so it won't affect production
df = df.drop("Mean Wind Speed")

In [ ]:
# Split months to summer and winter for irbid

from pyspark.sql.functions import col, when, year, count

# Convert the "Date" column to a date type
df = df.withColumn("Date", col("Date").cast("date"))

# Define the conditions for winter and summer seasons
winter_condition = (month(col("Date")).between(11, 12)) | (month(col("Date")).between(1, 5))
summer_condition = ~(winter_condition)

# Create new column "Season" based on the defined conditions
df = df.withColumn("Season", when(winter_condition, "Winter").otherwise("Summer"))

df.show()

In [ ]:
# Save the DataFrame as a CSV file to use with production data
df.write.csv('/content/Irbid_Cleansed_final.csv', header=True)

# Measuring Loading Time

In [ ]:
end_time = time.time()

loading_time = end_time - start_time
print("Loading Time:", loading_time, "seconds")

# Measuring Computing resources

In [ ]:
!pip install psutil

import psutil

cpu_usage = psutil.cpu_percent()
ram_usage = psutil.virtual_memory().percent

print("CPU Usage:", cpu_usage, "%")
print("RAM Usage:", ram_usage, "%")

# Measuring Storage resources

In [ ]:
!df -h